## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8287102 , -0.02134494, -0.5592708 ,  0.64586776],
        [ 0.16889122,  0.9622261 ,  0.21353377, -0.16179512],
        [ 0.5335871 , -0.27141353,  0.80101156,  0.09048137],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


In [8]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))

In [9]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [10]:
 gscene.NAME_DICT["floor"].dims = (2,1,0.01)

In [11]:
gscene.create_safe(GEOTYPE.BOX, "ceiling", "base_link", (2,1,0.01), 
                   (0.0,0.0,1.1),rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "leftwall", "base_link", (1,1,0.01), 
                   (1.0,0.0,0.5),rpy=(0,np.pi/2,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "rightwall", "base_link", (1,1,0.01), 
                   (-1.0,0.0,0.5),rpy=(0,np.pi/2,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "frontwall", "base_link", (2,1,0.01), 
                   (0.0,-0.5,0.5),rpy=(np.pi/2,0,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)

## add sweep face

In [12]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [13]:
gscene.update_markers_all()

## add wp

In [14]:
TOOL_DIM = [0.06, 0.10]
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    line = gscene.create_safe(GEOTYPE.BOX, "trc{}".format(i_trc+1), "base_link", (TRACK_DIM[0]*2, TRACK_DIM[0]*2, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=False, fixed=True, collision=False, parent="track_face")
    line_vis = gscene.create_safe(GEOTYPE.BOX, "trc{}vis".format(i_trc+1), "base_link", (TRACK_DIM[0], 0.01, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, line))

In [15]:
gscene.update_markers_all()

## add indy tool

In [16]:
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.07,0.07, 0.04), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts_col", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.11,0.11, 0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.03,0.03, 0.030), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.07,0.07, 0.030), rpy=(0,0,0), color=(0.0,0.8,0.0,0.2),
                            collision=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="indy0_brushbase", link_name="indy0_tcp",
                            center=(0,0,0.0775), dims=(0.06, 0.14, 0.015), rpy=(0,0,np.pi/4), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp",
                            center=(0,0,0.095), dims=(0.05,0.13, 0.015), rpy=(0,0,np.pi/4), color=(1.0,1.0,0.94,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp",
                            center=(0,0,0.080), dims=(0.08,0.16, 0.02), rpy=(0,0,np.pi/4), color=(0.0,0.8,0.0,0.5),
                            collision=True)

## add box

In [17]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [18]:
# give clearance from the track surface
CLEARANCE = 1e-3
Ttrack = track.get_tf(crob.home_dict)
Ttrack_inv = SE3_inv(Ttrack)
for gtem in gtem_dict.values():
    verts, _ = gtem.get_vertice_radius_from(crob.home_dict, from_link="base_link")
    verts_loc = np.matmul(Ttrack_inv[:3,:3], verts.transpose())+Ttrack_inv[:3,3:]
    off = track.dims[2]/2-np.min(verts_loc[2,:])+CLEARANCE
    center_loc = np.matmul(Ttrack_inv[:3,:3], gtem.center) + Ttrack_inv[:3,3]
    center_loc_new = center_loc + [0,0,off]
    center_new = np.matmul(Ttrack[:3,:3], center_loc_new) + Ttrack[:3,3]
    gtem.set_offset_tf(center=center_new)

In [19]:
gscene.update_markers_all()

## Register binders

In [20]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [21]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))

## add objects

In [22]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [23]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_object(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [24]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [25]:
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, tline = track_tem
    sweep1 = pscene.create_object(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                 action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])},
                                geometry_vertical = tline)

In [26]:
# pscene.remove_object("sweep3")

### planners

In [27]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [28]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [29]:
mplan.motion_filters = checkers_all

In [30]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [31]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track_face', 'track_face', 0, 0, 0)


In [32]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [33]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler

In [34]:
##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class GrowingSampler:
    def __init__(self, log2_decay=1):
        self.log2_decay = log2_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        for node in nodes:
            if node not in self.log2_prob_dict:
                self.log2_prob_dict[node] = 1.0
        probs = [np.exp2(self.log2_prob_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
#             probs = np.ones_like(probs)/len(probs)
            with self.prob_lock:
                for k in self.node_dict.keys():
                    self.log2_prob_dict[k] = 1.0
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node

    
##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class PenaltyNodeSampler:
    def __init__(self, log2_decay=1, log2_count_decay=1):
        self.log2_decay = log2_decay
        self.log2_count_decay = log2_count_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.log2_count_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.log2_count_dict = dict()
            self.prob_lock = DummyBlock()

        for k in self.node_dict.keys():
            self.log2_prob_dict[k] = 1.0
            self.log2_count_dict[k] = 0.0

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [np.exp2(self.log2_prob_dict[node]+self.log2_count_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
#             probs = np.ones_like(probs)/len(probs)
            with self.prob_lock:
                for k in self.node_dict.keys():
                    self.log2_prob_dict[k] = 1.0
                    self.log2_count_dict[k] = 0.0
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node
    
    def update(self, snode_src, snode_new, connection_result):
        if connection_result:
            self.log2_count_dict[snode_new.state.node] = self.log2_count_dict[snode_new.state.node] - self.log2_count_decay
            

In [35]:
tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [36]:
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [37]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        self.NUM_WP_TRIALS = 1
        
    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.call_count_dict = multiprocess_manager.dict()
            self.count_lock = multiprocess_manager.Lock()
        else:
            self.call_count_dict = dict()
            self.count_lock = DummyBlock()
            
        no_enter_sidxes = [stype==SweepLineTask for stype in enumerate(self.pscene.subject_type_list)]
        no_enter_initials = [tplan.initial_state.node[sidx] for sidx in no_enter_sidxes]
        self.sorted_unstop_keys = sorted(tplan.unstoppable_terminals.keys())
            
        self.node_dict={}
        self.enter_dict={}
        for node, leafs in tplan.node_dict.items():
            self.node_dict[node] = deepcopy(leafs)
            if not all([node[k] in terms for k, terms in tplan.unstoppable_terminals.items()]):
                tplan.node_dict[node] = set([]) ## unstoppable node change will be reserved by this custom rule
            else:
                # entering to unstoppable terminal is controlled by this rule
                self.enter_dict[node] = set([leaf for leaf in leafs 
                                             if any([leaf[k] not in terms for k, terms in tplan.unstoppable_terminals.items()])
                                             and all([leaf[k]>=leaf[k+1] for k in self.sorted_unstop_keys[:-1]])]) # the task is done in order
                tplan.node_dict[node] = set([leaf for leaf in leafs 
                                             if all([leaf[k] in terms for k, terms in tplan.unstoppable_terminals.items()])
                                            ])

        self.node_parent_dict={}
        for node, parents in tplan.node_parent_dict.items():
            self.node_parent_dict[node] = deepcopy(parents)
            tplan.node_parent_dict[node] = parents = set(
                [parent for parent in parents ## unstoppable node change will be reserved by this custom rule
                 if all([parent[k] in terms for k, terms in tplan.unstoppable_terminals.items()])])
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
#         print("CustomRule call")
        stack_res = False
        stack_items = []
        if connection_result and snode_new.state.node in self.enter_dict:
            with tplan.snode_dict_lock:
                nb_nodes = list(tplan.neighbor_nodes.keys())
                for nb_node in nb_nodes:
                    for ukey in self.sorted_unstop_keys:
                        if nb_node[ukey] < snode_new.state.node[ukey]:
                            del tplan.neighbor_nodes[nb_node]
                            break # task is done in order
        
        if connection_result and snode_new.state.node in self.enter_dict:
            stack_res, stack_items = True, list(self.enter_dict[snode_new.state.node])
            
        if snode_src is not None:
            with self.count_lock:
                if snode_src.idx in self.call_count_dict:
                    self.call_count_dict[snode_src.idx] = self.call_count_dict[snode_src.idx] + [snode_new.state.node]
                else:
                    self.call_count_dict[snode_src.idx] = [snode_new.state.node]
            
            node_src = snode_src.state.node
            node_new = snode_new.state.node
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(node_src, node_new)])[0]
#             print("{}->{} , diff: {}".format(node_src, node_new, diff_sidxes))
            if len(diff_sidxes)==0:
                return stack_res, stack_items
            diff_sidx = diff_sidxes[0]
            diff_sname = self.pscene.subject_name_list[diff_sidx]
            diff_subject = self.pscene.subject_dict[diff_sname]
            
            if isinstance(diff_subject, SweepLineTask):
#                 print("Rule for SweepLineTask")
                with tplan.snode_dict_lock:
                    if node_src[diff_sidx] not in tplan.unstoppable_terminals[diff_sidx]: # from intermediate wp -> remove access to them
                        snode_list = tplan.node_snode_dict[node_src]
                        if snode_src.idx in snode_list:
#                             print("Removing {} from {}: {} not in {}".format(snode_src.idx, snode_list, node_src[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
#                             print(snode_list)
                            snode_list.remove(snode_src.idx)
                            tplan.node_snode_dict[node_src] = snode_list
                            print(tplan.node_snode_dict[node_src])
                        if len(snode_list) == 0 and node_new in tplan.neighbor_nodes:
                            del tplan.neighbor_nodes[node_new]
                    if connection_result:
                        if node_new[diff_sidx] in tplan.unstoppable_terminals[diff_sidx]: ## in terminal condition
#                             print("Check home: {} in {}".format(node_new[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            link_name = self.pscene.gscene.NAME_DICT[snode_new.state.binding_state[diff_sidx][-1]].link_name
                            rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                            link_name in chain_vals['link_names']]
                            if len(rname_candis)==0:
                                print("no robot candis")
                            else:
#                                 print("reserve homing = {}".format(snode_new.idx))
                                newstate = snode_new.state.copy(self.pscene)
                                jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                                newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                                return True, [newstate] + stack_items
                        else:## not in terminal condition
#                             print("Reserve next wp from {}".format(snode_new.idx))
                            snode_list = tplan.node_snode_dict[node_new]
                            if snode_new.idx in snode_list:
#                                 print("Removing {} from {} as next wp is reserved".format(snode_new.idx, snode_list))
                                snode_list.remove(snode_new.idx)
                                tplan.node_snode_dict[node_new] = snode_list
                            next_node_candis = list(self.node_dict[node_new])
                            assert len(next_node_candis) == 1, "non-terminal sweep task should have only 1 leaf ({}) {}-{}".format(diff_sidx, node_new, next_node_candis)
                            return True, next_node_candis*self.NUM_WP_TRIALS + stack_items
                return stack_res, stack_items
            
            if isinstance(diff_subject, AbstractObject):
#                 print("Rule for AbstractObject")
                if connection_result:
                    link_name1 = snode_src.state.state_param[diff_sname][0]
                    link_name2 = snode_new.state.state_param[diff_sname][0]
                    rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                   link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                    if len(rname_candis)==0:
                        print("no robot candis")
                        return stack_res, stack_items
                    else:
#                         print("try homing")
                        newstate = snode_new.state.copy(self.pscene)
                        jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                        newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                        return True, [newstate] + stack_items
        return stack_res, stack_items

In [38]:
tplan.custom_rule = CustomRule(pscene)

In [39]:
mplan.reset_log(True)

In [40]:
gtimer.reset()
# goal_nodes = [("track_face","track_face")+(2,0,0)]
goal_nodes = [("track_face","track_face")+tuple([2 for _ in range(TRACK_NUM)])]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=200, multiprocess=True, timeout=10, timeout_constrained=50)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
transition motion tried: True
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = success
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
branching: 0->1 (0.17/200.0 s, steps/err: 15(83.5788249969 ms)/0.00127692909362)
try: 1 - ('track_face', 'track_face', 1, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('

transition motion tried: True
try: 8 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 6 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = success
branching: 6->10 (0.95/200.0 s, steps/err: 15(49.5829582214 ms)/0.00171447827776)
try: 8 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
joint motion tried: True
result: 4 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = success
branching: 4->11 (1.02/200.0 s, steps/err: 23(291.042089462 ms)/8.81487848828e-16)
try: 9 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
try: 10 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
joint motion tried: True
result: 8 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
branching: 8->12 (1.07/200.0 s, steps/err: 33(135.229110718 ms)/2.34194453275e-16)
try: 12 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
try: 11 - ('grip1', 'tr

result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 14 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 4 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
branching: 4->14 (1.85/200.0 s, steps/err: 10(388.589859009 ms)/0.00139210146441)
try: 14 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 11 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
transition motion tried: True
result: 11 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 14 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 15 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
transition motion tried: True
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
result: 13 - ('track_face', 'grip1', 0, 0, 0)->('track_fa

try: 29 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
transition motion tried: True
result: 28 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = success
branching: 28->31 (3.59/200.0 s, steps/err: 15(124.346017838 ms)/0.00176832589706)
try: 29 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
transition motion tried: True
result: 29 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = success
branching: 29->32 (3.89/200.0 s, steps/err: 15(213.422060013 ms)/0.00185452652361)
try: 30 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
transition motion tried: True
result: 30 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = success
branching: 30->33 (4.09/200.0 s, steps/err: 15(121.467113495 ms)/0.00171605262009)
try: 31 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
joint motion tried: True
result: 29 - ('track_face', 

result: 39 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 39 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 36 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 37 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 37 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 39 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 37 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 37 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 35 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 37 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 36 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 39 - ('track_face', 'grip1', 2

...

branching: 284->292 (58.97/200.0 s, steps/err: 36(628.02696228 ms)/9.10723483204e-16)
transition motion tried: True
transition motion tried: True
try: 291 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
result: 281 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = success
result: 287 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = success
branching: 287->294 (59.06/200.0 s, steps/err: 12(283.462047577 ms)/0.00216038970963)
branching: 281->293 (59.07/200.0 s, steps/err: 13(1262.22395897 ms)/0.00153500971814)
result: 291 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 293 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2)
try: 292 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
joint motion tried: True
try: 294 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2)
result: 288 - ('grip1', 'track_face', 2, 2, 0)->('grip1

try: 305 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
joint motion tried: True
result: 304 - ('grip1', 'track_face', 2, 2, 2)->('grip1', 'track_face', 2, 2, 2) = success
branching: 304->306 (61.57/200.0 s, steps/err: 11(91.5110111237 ms)/9.48864120849e-16)
try: 306 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 306 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
try: 306 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 306 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
try: 304 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 304 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
try: 304 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 304 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
try: 306 - ('gr

branching: 306->308 (63.01/200.0 s, steps/err: 27(899.398088455 ms)/0.00141248143006)
======================= terminated 11: first answer acquired ===============================
transition motion tried: True
result: 306 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = success
branching: 306->309 (63.17/200.0 s, steps/err: 17(719.40112114 ms)/0.00178549737044)
transition motion tried: True
result: 281 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
======================= terminated 10: first answer acquired ===============================
branching: 281->310 (63.24/200.0 s, steps/err: 38(3482.97595978 ms)/0.00190317131822)
======================= terminated 12: first answer acquired from other agent ===============================
transition motion tried: False
result: 259 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = fail
constrained motion tried: True
======================= terminated 16: first 

result: 293 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2) = success
branching: 293->317 (71.24/200.0 s, steps/err: 49(12125.977993 ms)/0.00162957986641)
======================= terminated 1: first answer acquired from other agent ===============================
transition motion tried: False
result: 305 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = fail
======================= terminated 19: first answer acquired from other agent ===============================
constrained motion tried: True
result: 226 - ('track_face', 'grip1', 2, 1, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 226->319 (75.84/200.0 s, steps/err: 53(25404.1540623 ms)/0.00162838470335)
======================= terminated 4: first answer acquired from other agent ===============================


Process Process-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 153, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 234, in plan_algorithm
    group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=timeout)
  File "pkg/planning/motion/moveit/moveit_py.py", line 113, in plan_py
    JointState(self.joint_num, *Q_init), plannerconfig, timeout)
KeyboardInt

In [41]:
print(gtimer)

plan: 	62835.0 ms/1 = 62834.89 ms (62834.89/62834.89)



In [44]:
# for i, sidx in enumerate([1, 2, 4, 6]):
#     save_json("traj{}.json".format(i), tplan.snode_dict[sidx].traj)
#     gscene.show_motion(tplan.snode_dict[sidx].traj, period=0.05)
        

In [45]:
# import matplotlib.pyplot as plt
# kin = stereo.camera_list[0]
# rs = stereo.camera_list[1]

In [46]:
# plt.figure(figsize=(15,10))
# plt.imshow(kin.get_image())

In [47]:
# plt.figure(figsize=(15,10))
# plt.imshow(rs.get_image())

## play searched plan

In [42]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [43]:
snode_schedule = ppline.add_return_motion(snode_schedule)

joint motion tried: True


In [44]:
ppline.play_schedule(snode_schedule, period=0.03)

In [45]:
# print("schedule length: {}".format(len(snode_schedule)))
# for i_s,  snode in enumerate(snode_schedule):
#     print("{}: {}".format(i_s, snode.state.node))

In [46]:
# save_json("traj.json", snode_schedule[1].traj)

## execute plan

In [57]:
crob.reset_connection([True,True])

connection_list
[True, True]
Connect: Server IP (192.168.0.63)


In [58]:
indy = crob.robot_dict["indy0"]
panda = crob.robot_dict["panda1"]
with indy:
    indy.set_joint_vel_level(1)
    indy.set_task_vel_level(1)

Connect: Server IP (192.168.0.63)


In [59]:
crob.joint_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)


In [60]:
indy.stop_tracking()

{'stop': True}

In [52]:
crob.grasp_by_dict({"indy0":False, "panda1": False})

grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)


In [53]:
ppline.execute_schedule(snode_schedule, vel_scale=0.3, acc_scale=0.05)

binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)
go


KeyboardInterrupt: 

In [ ]:
for i_s, snode in enumerate(snode_schedule[1:]):
    save_json("traj%02d.json"%(i_s), snode.traj)

In [ ]:
indy = crob.robot_dict["indy0"]
indy.stop_tracking()

## MUST CHECK! no more than 1 trial from non-terminal nodes
## MUST CHECK! no more than 1 trial to same framed motion
## MUST CHECK! efficient binding/param sampling for gripper

In [ ]:
for k, v in tplan.custom_rule.call_count_dict.items():
    print("{}: {} - {}".format(k,tplan.snode_dict[k].state.node, sorted(v)))

## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [ ]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

## extend preserving goal-matching items

In [ ]:
print(gtimer)

## extend only no reservation

In [ ]:
print(gtimer)

## no extension

In [ ]:
print(gtimer)

## extend_toward goal

In [ ]:
print(gtimer)